# RETO 1 - "Diabetes 130-US hospitals for years 1999-2008"

It is important to mention that to have a good context, reading this paper is necessary because
multiple analyses were made based on the information provided there:

**"https://www.hindawi.com/journals/bmri/2014/781670/"**

In [1]:
# Packages

import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

## Importing the data

In [2]:
# Install in the local machine the package "repository" of the datasets 
# from the UC Irvine Machine Learning Repository

!pip3 install -U -q ucimlrepo

In [3]:
# Import the specific dataset from the repository

from ucimlrepo import fetch_ucirepo

# We use warnings to prevent any warning message from being displayed because
# we will check the dataframes in detail on the next steps
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # fetch dataset 
    diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296)
  
    # data (as pandas dataframes) 
    features = diabetes_130_us_hospitals_for_years_1999_2008.data.features 
    target = diabetes_130_us_hospitals_for_years_1999_2008.data.targets

In [4]:
# Quick view to check the dataframes (features)
features.head()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,No
1,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
2,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,NaN,...,No,No,No,No,No,No,No,No,No,Yes
3,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
4,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,...,No,No,Steady,No,No,No,No,No,Ch,Yes


In [5]:
# Quick view to check the dataframes (target)
target.head()

,readmitted
0,NO
1,>30
2,NO
3,NO
4,NO


## Data exploration

#### Features

In [6]:
# Dimensions of the dataset
features.shape

(101766, 47)

In [7]:
# Let's see the non-null count per column and the Dtype
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 47 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   race                      99493 non-null   object
 1   gender                    101766 non-null  object
 2   age                       101766 non-null  object
 3   weight                    3197 non-null    object
 4   admission_type_id         101766 non-null  int64 
 5   discharge_disposition_id  101766 non-null  int64 
 6   admission_source_id       101766 non-null  int64 
 7   time_in_hospital          101766 non-null  int64 
 8   payer_code                61510 non-null   object
 9   medical_specialty         51817 non-null   object
 10  num_lab_procedures        101766 non-null  int64 
 11  num_procedures            101766 non-null  int64 
 12  num_medications           101766 non-null  int64 
 13  number_outpatient         101766 non-null  int64 
 14  numb

In [8]:
# % Missing data
percentage_missing_values = (features.isna().sum()/features.shape[0])*100
percentage_missing_values

race                         2.233555
gender                       0.000000
age                          0.000000
weight                      96.858479
admission_type_id            0.000000
discharge_disposition_id     0.000000
admission_source_id          0.000000
time_in_hospital             0.000000
payer_code                  39.557416
medical_specialty           49.082208
num_lab_procedures           0.000000
num_procedures               0.000000
num_medications              0.000000
number_outpatient            0.000000
number_emergency             0.000000
number_inpatient             0.000000
diag_1                       0.020636
diag_2                       0.351787
diag_3                       1.398306
number_diagnoses             0.000000
max_glu_serum                0.000000
A1Cresult                    0.000000
metformin                    0.000000
repaglinide                  0.000000
nateglinide                  0.000000
chlorpropamide               0.000000
glimepiride 

###### Handling missing values and changing dtypes

In [9]:
# Columns with a high number (>=30%) of missing values
columns_to_delete = percentage_missing_values[percentage_missing_values >= 30]

# Remove column "medical_specialty" from the serie
del columns_to_delete["medical_specialty"]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # Delete columns from dataframe
    features.drop(columns=columns_to_delete.index, inplace=True)

    # Columns with missing values to impute
    column_missing_values = features.isna().sum()[features.isna().sum() > 0]

    # Columns to impute filling with a Unique Category
    column_missing_values.pop("medical_specialty")
    features["medical_specialty"].fillna("missing", inplace=True)

    # Imputing data using the mode
    features[column_missing_values.index] = features[column_missing_values.index].apply(lambda x: x.fillna(x.mode().iloc[0]))

In [10]:
# Changing the dtypes
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    features["admission_type_id"] = features["admission_type_id"].astype(str)
    features["discharge_disposition_id"] = features["discharge_disposition_id"].astype(str)
    features["admission_source_id"] = features["admission_source_id"].astype(str)

###### Descriptive Statistics 

In [11]:
features.describe()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


In [12]:
features.select_dtypes(include='object').nunique()

race                          5
gender                        3
age                          10
admission_type_id             8
discharge_disposition_id     26
admission_source_id          17
medical_specialty            73
diag_1                      716
diag_2                      748
diag_3                      789
max_glu_serum                 4
A1Cresult                     4
metformin                     4
repaglinide                   4
nateglinide                   4
chlorpropamide                4
glimepiride                   4
acetohexamide                 2
glipizide                     4
glyburide                     4
tolbutamide                   2
pioglitazone                  4
rosiglitazone                 4
acarbose                      4
miglitol                      4
troglitazone                  2
tolazamide                    3
examide                       1
citoglipton                   1
insulin                       4
glyburide-metformin           4
glipizid

#### Cleaning and aggregating variables with many catogories

In [13]:
# Race will be in three categories to balance the data

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    features["race"] = features["race"].replace(["Hispanic", "Asian"], "Other")

In [14]:
# Gender will have only two categories "Female" and "Male"
# because there are just 3 "unknown" observations

gender_mask = (features["gender"] != "Unknown/Invalid")

features = features[gender_mask]
target = target[gender_mask]

In [15]:
# Age will be aggregated into 3 categories (trying to split the ranges proportionally):

younger_30 = ["[20-30)", "[10-20)", "[0-10)"]
older_30_younger_60 = ["[40-50)", "[30-40)", "[50-60)"]
older_60 = ["[70-80)", "[60-70)", "[80-90)", "[90-100)"]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    features['age'] = features['age'].replace(younger_30, "<=30")
    features['age'] = features['age'].replace(older_30_younger_60, "(30-60)")
    features['age'] = features['age'].replace(older_60, ">=60")

In [16]:
# Medical_specialty will be divided into 5 categories (including missing) and adding a new one "other"
print(features["medical_specialty"].value_counts().sort_values(ascending=False)[:4])

categories_to_other = features["medical_specialty"].value_counts().sort_values(ascending=False)[4:].index

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    features["medical_specialty"] = features["medical_specialty"].replace(categories_to_other, "other")

missing                   49947
InternalMedicine          14635
Emergency/Trauma           7565
Family/GeneralPractice     7440
Name: medical_specialty, dtype: int64


In [17]:
# diag_1, diag_2, diag_3. For these columns, we aggregated using the encoded "ICD9", in the way the paper's author suggests

disease_circulatory_system = [str(x) for x in (list(range(390,460)) + [785])]
diabetes = pd.Series([x for x in features["diag_1"] if x[:3] == "250"]).unique()
disease_respiratory_system = [str(x) for x in (list(range(460,520)) + [786])] 
diseases_digestive_system = [str(x) for x in (list(range(520,580)) + [787])]
injury_and_poisoning = [str(x) for x in (list(range(800,1000)))]
diseases_musculoskeletal_system_connective_tissue = [str(x) for x in (list(range(710,740)))]
diseases_genitourinary_system = [str(x) for x in (list(range(580,630)) + [788])]
neoplasms = [str(x) for x in (list(range(140,239)))]
categories_used = ["disease_circulatory_system", "diabetes", "disease_respiratory_system", 
                   "diseases_digestive_system", "injury_and_poisoning", "diseases_musculoskeletal_system_connective_tissue",
                   "diseases_genitourinary_system", "neoplasms"]
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for x in ['diag_1', 'diag_2', 'diag_3']:
        features[x] = features[x].replace(disease_circulatory_system, "disease_circulatory_system")
        features[x] = features[x].replace(diabetes, "diabetes")
        features[x] = features[x].replace(disease_respiratory_system, "disease_respiratory_system")
        features[x] = features[x].replace(diseases_digestive_system, "diseases_digestive_system")
        features[x] = features[x].replace(injury_and_poisoning, "injury_and_poisoning")
        features[x] = features[x].replace(diseases_musculoskeletal_system_connective_tissue,
                                                    "diseases_musculoskeletal_system_connective_tissue")
        features[x] = features[x].replace(diseases_genitourinary_system, "diseases_genitourinary_system")
        features[x] = features[x].replace(neoplasms, "neoplasms")
        other = pd.Series([i for i in features[x] if i not in categories_used]).unique()
        features[x] = features[x].replace(other, "other")

In [18]:
# For each of the columns "admission_type_id", "discharge_disposition_id" and "admission_source_id"
# will be 4 categories to balance the data. The top 3 frequent categories and the four one will be labeled as "Other"
# to sum up the order records.

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for x in ["admission_type_id","discharge_disposition_id","admission_source_id"]:
        top_3 = features[x].value_counts().sort_values(ascending=False)[0:3].index
        other = pd.Series([i for i in features[x] if i not in top_3]).unique()
        features[x] = features[x].replace(other, "other") 

#### Target

In [19]:
# Dimensions of the dataset
target.shape

(101763, 1)

In [20]:
# Unique values
target["readmitted"].unique()

array(['NO', '>30', '<30'], dtype=object)

In [21]:
# missing values
target.isna().sum()

readmitted    0
dtype: int64

In [22]:
# Number of records by readmitted category
target.value_counts()

readmitted
NO            54861
>30           35545
<30           11357
dtype: int64

## Experimentation

In this phase we will play around  with the data in order to find the best possible solution for the inquiring:

**"What variables are significant to determine the readmittion of a patient?"**

**"How can we improve this readmittion rate?"**

All the variables in this dataset have an impact on analyzing the readmittion of a patient
because they were selected by a group of health experts, so we trust in their decision. 
Then, our goal will be to determine **the most significant** variables to predict the readmittion and give recommendations.

##### Convert categorical variables into a numerical format for fitting the model. Using one-hot encoding

In [23]:
features_encoded = pd.get_dummies(features)
target_encoded = pd.get_dummies(target)

### Split into training and testing sets

In [24]:
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target_encoded, test_size=0.3, stratify=target, random_state=12)

### Model to make predictions - Random Forest -

In [25]:
rf_selection_model = RandomForestClassifier(n_estimators=500, random_state=12, n_jobs=-1)
rf_selection_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=12)

In [26]:
# Predictions with train set
y_pred_train = rf_selection_model.predict(X_train)

In [27]:
print("F1 score:", f1_score(y_pred_train, y_train, average="weighted"))

F1 score: 1.0


In [28]:
# Predictions with test set
y_pred_test = rf_selection_model.predict(X_test)

In [29]:
print("F1 score:", f1_score(y_pred_test, y_test, average="weighted"))

F1 score: 0.6157909192661022


##### Lets variate the hyperparameters of the model to improve its performance

In [30]:
rf_selection_model_adjusted = RandomForestClassifier(n_estimators=500,
                                                     max_leaf_nodes=16, # parameter to reduce overfitting
                                                     max_depth = 4, # parameter to reduce overfitting
                                                     random_state=12,
                                                     n_jobs=-1)
rf_selection_model_adjusted.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, max_leaf_nodes=16, n_estimators=500,
                       n_jobs=-1, random_state=12)

In [31]:
# Predictions with test set
y_pred_test_adjusted = rf_selection_model_adjusted.predict(X_test)

In [32]:
print("F1 score:", f1_score(y_pred_test_adjusted, y_test, average="weighted"))

F1 score: 0.7016231618582405


#### Feature selection

In [33]:
# Create an ordered serie sorted by relevant features in the model

feature_importance = {name: score for name, score in zip(list(features_encoded), rf_selection_model_adjusted.feature_importances_)}
feature_importance_sorted = pd.Series(feature_importance).sort_values(ascending=False)

In [64]:
# Extract the 40 most relevant features
columns = list(feature_importance_sorted.head(40).index)
X_train_reduced = X_train[columns].copy()
X_test_reduced = X_test[columns].copy()

In [65]:
# Retrain the model with relevant features
rf_selection_model_reduced = RandomForestClassifier(n_estimators=500,
                                                    random_state=12,
                                                    max_leaf_nodes=16, 
                                                    max_depth = 4,
                                                    n_jobs=-1)
rf_selection_model_reduced.fit(X_train_reduced, y_train)

RandomForestClassifier(max_depth=4, max_leaf_nodes=16, n_estimators=500,
                       n_jobs=-1, random_state=12)

In [66]:
# Predictions with train reduce set
y_pred_train_reduced = rf_selection_model_reduced.predict(X_train_reduced)

In [67]:
print("F1 score:", f1_score(y_pred_train_reduced, y_train, average="weighted"))

F1 score: 0.6980753743588073


In [68]:
# Predictions with test reduce set
y_pred_test_reduced = rf_selection_model_reduced.predict(X_test_reduced)

In [69]:
print("F1 score:", f1_score(y_pred_test_reduced, y_test, average="weighted"))

F1 score: 0.6936289823746131


## Conclusions

In [70]:
feature_importance_sorted.head(10)

number_inpatient                  0.297100
number_emergency                  0.116495
number_diagnoses                  0.085224
number_outpatient                 0.078223
admission_source_id_other         0.060723
num_medications                   0.041304
admission_source_id_7             0.036984
discharge_disposition_id_other    0.035510
diabetesMed_Yes                   0.022509
discharge_disposition_id_6        0.021677
dtype: float64